### Creating multi-agent collaboration: Hedge Fund assistant
---

In this folder we will create the supervisor agent for our `Hedge Fund assistant` and attach the `fundamental analyst assistant`, `technical analyst assistant` and `marketing analyst assistant` as sub-agents to it.

**IMPORTANT: This notebook uses the new `Amazon Nova Micro` model as the FM that powers the supervisor agent to route requests efficiently between associated sub agents.**

Each of the sub agents have access to three tools each that can access `APIs` to fetch financial data on ticker symbols and search the web. Based on the user question, the supervisor agent hedge assistant will be able to call the appropriate sub agent. The sub agent will then identify the question and call one of the functions that it has access to, and return the answer back to the user. View the diagram of what this looks like below:

![multi-agent-diagram](/multi-agent-diagram.png)

This agent will make usage of the `SUPERVISOR_ROUTER` collaboration mode in order to get the user's intent classification routed to the correct sub-agent

### Agent invocation tracking
---

As a part of the `invoke_agent_helper` function, we have integrated tracking on `weave`. For this, enter your `WEAVE_API_KEY` in the `.env` file. View an example below:

![weave-tracking](weave-tracking.png)

### Prerequisites
---

You are going to install boto3 dependencies from pip. Make sure you have the latest version of it for full capabilities

Before running this notebook, make sure to populate the `.env` file with your `FINANCIAL_DATASETS_API_KEY` and `TAVILY_API_KEY`. These two `API` keys will be used by the agent to access information on user related questions. You can create an `API` here: https://www.financialdatasets.ai/

In [ ]:
!pip uninstall boto3 botocore awscli --yes

In [ ]:
# Install latest boto3
!python3 -m pip install --force-reinstall --no-cache -q --no-dependencies -r ../requirements.txt

### Import required libraries
---

Next we will import the required libraries. We will also import some support functions available in the parent directory. Those functions are:

1. `create_agent`: helps you to create the necessary IAM permissions and Bedrock agents based on the agent's name, instructions, foundation models, descriptions and other properties.

1. `associate_sub_agents`: associate existing agents as sub-agents to the created supervisor agent

1. `list_agent_collaborators`: list the available agents collaborations

1. `invoke_agent_helper`: helps you to invoke your agent using invoke_agent
You can see the implementation of both functions in the parent directory

In [ ]:
import os
import sys
import json
import time
import logging

# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)

from globals import *
from agents import create_agent, associate_sub_agents, list_agent_collaborators, invoke_agent_helper

In [ ]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

#### Defining supervisor agent configuration

Let's now define the configuration for our hedge fund assistant supervisor agent. Let's use the follow instructions to create our agent:

You are a hedge fund assistant who fetches information on fundamental, technical and marketing analysis on financial data. You can fetch this different kind of financial data based on user questions.

In [ ]:
%store -r
fundamental_analyst_agent_alias_arn, marketing_agent_alias_arn, technical_analyst_agent_alias_arn

In [ ]:
agent_instruction: str = """You are a sophisticated financial analysis system with access to three specialized analysts: a Fundamental Analyst who analyzes financial statements, balance sheets, and cash flows; 
a Technical Analyst who examines price patterns, technical indicators, and market trends; and a Sentiment Analyst who monitors market sentiment through options data, insider trading, and news coverage. When responding to user queries, 
coordinate with these analysts to provide comprehensive investment insights. The Fundamental Analyst will focus on company financials and valuation metrics, the Technical Analyst will analyze price movements and technical indicators, 
and the Sentiment Analyst will gauge market psychology and news sentiment. Work together to synthesize their individual analyses into a cohesive, well-rounded response that considers multiple perspectives on the investment opportunity. 
Each analysis should be clearly labeled and contribute to an overall recommendation or insight."""

In [ ]:
agent_name = 'multi-agent-hedge-fund'
agent_description = "Multi-agent collaboration for hedge fund assistant to get fundamental, technical and marketing analysis"

sub_agents_list = [
    {
        'sub_agent_alias_arn': fundamental_analyst_agent_alias_arn,
        'sub_agent_instruction': """Use this agent when the user asks to analyze company financial statements across three key areas: income statements, balance sheets, and cash flow statements. You require the user to provide a stock ticker symbol to analyze the company's financials. If a user does not provide a ticker symbol, please ask for it as a follow-up question.""",
        'sub_agent_association_name': 'fundamental-analyst-agent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': marketing_agent_alias_arn,
        'sub_agent_instruction': """Use this agent when a user asks to analyze options chains, insider trading data, and relevant market news. You require the user to provide a stock ticker symbol for analysis. If a user does not provide a ticker symbol, mention in the answer that they need to provide a ticker symbol.""",
        'sub_agent_association_name': 'market-analysis-agent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': technical_analyst_agent_alias_arn,
        'sub_agent_instruction': """Use this agent if a users asks to analyze stock price movements and technical indicators. You can calculate and interpret various technical indicators using historical price data.""",
        'sub_agent_association_name': 'technical-analyst-agent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    }
]

### Create the hedge fund agent
---

Next we will create the agent with the provided information

In [ ]:
supervisor_agent_id, supervisor_agent_alias_id, supervisor_agent_alias_arn = create_agent(
    agent_name,
    agent_instruction,
    agent_foundation_model=BEDROCK_NOVA_MICRO,
    agent_description=agent_description,
    agent_collaboration='SUPERVISOR',
    create_alias=False
)
supervisor_agent_id, supervisor_agent_alias_id, supervisor_agent_alias_arn

### Create agent associations
---

Next we will associate the required sub-agents to our supervisor agent

In [ ]:
supervisor_agent_alias_id, supervisor_agent_alias_arn = associate_sub_agents(
    supervisor_agent_id, sub_agents_list
)
supervisor_agent_alias_id, supervisor_agent_alias_arn

Now you would have the four agents created and prepared on the console - this includes the `SUPERVISOR HEDGE FUND AGENT` and 3 `SUB AGENTS`

![agents](agents-created.png)

### List agent associations
---

After that we can list the associations of our agent

In [ ]:
logger.info(f"The collaborators of the multi hedge fund agent are: {list_agent_collaborators(supervisor_agent_id)}")
logger.info(f"Number of agent collaborators: {len(list_agent_collaborators(supervisor_agent_id))}")

### Testing Agent
---

Now that we've created the agent, let's test it by using our invoke_agent_helper function

In [ ]:
time.sleep(10)

#### Route to the Marketing analyst sub agent
---

In this portion, we will invoke the supervisor agent to look for General Council's stocks in `AAPL`. For this, the `marketing analyst agent` should be invoked.

The `get_news()` function is used by the `sub-agent` to get the latest information on the question below.

In [ ]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """How many shares of AAPL has General Counsel sold over the past year?"""
response = invoke_agent_helper(query, session_id, supervisor_agent_id, supervisor_agent_alias_id, enable_trace=True)
print(response)

#### Route to the Fundamental analyst agent
---

In this portion, we will invoke the supervisor agent to look for AAPL's balance sheet information for the first quarter of May, 2021. For this, the supervisor agent should route the request to the `fundamental analyst sub agent` and get the information.

The `get_balance_sheets()` function is used by the sub agent to get the response below

In [ ]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """What is AAPL's balance sheet information for 2023?"""
response = invoke_agent_helper(query, session_id, supervisor_agent_id, supervisor_agent_alias_id, enable_trace=True)
print(response)

### Route to the `Marketing analyst agent` to get news by searching on web
---

In [ ]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """Can you give any news on 'AAPL' and what is its stock like? Search for 'Apple stock data'. Look for statistical data to provide. Be concise and only search once."""
response = invoke_agent_helper(query, session_id, supervisor_agent_id, supervisor_agent_alias_id, enable_trace=True)
print(response)

### Invoking collaboration between two sub agents via one user query
---

In this example, we will ask for some news as well as balance sheet information which will require the supervisor agent to collaborate with both, the marketing analyst agent and the fundamental analyst agent.

In [ ]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """Search for 'Apple stock data' and give the latest news. Also, what is AAPL's balance sheet information for 2023?"""
response = invoke_agent_helper(query, session_id, supervisor_agent_id, supervisor_agent_alias_id, enable_trace=True)
print(response)